### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let\'s start by recalling what I know about parrots. They\'re birds known for their ability to mimic human speech. But why do they do that? I remember reading somewhere that it\'s related to their social behavior. Maybe they imitate sounds to communicate with others? But why specifically human speech?\n\nFirst, I should consider their natural environment. Parrots are social creatures, right? They live in flocks. In the wild, they might use calls to communicate with each other, warn about predators, or find mates. If they can mimic sounds, including human speech, it might be an extension of that communication need. But domesticated parrots learn to talk around humans. So perhaps they see humans as part of their social group and try to communicate by imitating our sounds.\n\nAnother angle is their intelligence. Parrots are considered highly intelligent birds. They have a part of the brain that\'s similar to the h

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments.\n", 'tool_calls': [{'id': 'c6smtcmmb', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 154, 'total_tokens': 240, 'completion_time': 0.160528624, 'completion_tokens_details': {'reasoning_tokens': 62}, 'prompt_time': 0.005997383, 'prompt_tokens_details': None, 'queue_time': 0.057710677, 'total_time': 0.166526007}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--e7bb7c48-663e-47ad-b

### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny.


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. Let me check the function parameters. The required parameter is location, which should be a string. Boston is the location here. So I\'ll call the function with location set to "Boston". Make sure the JSON is correctly formatted with the function name and arguments. No other functions are available, so this should be straightforward.\n', 'tool_calls': [{'id': '20cfda4p0', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 153, 'total_tokens': 262, 'completion_time': 0.199966855, 'completion_tokens_details': {'reasoning_tokens': 85}, 'prompt_time': 0.006519155, 'prompt_tokens_details': None, 'queue_time': 0.055720395, 'total_time': 0.20648601}, 'mod